Author: Omar El Malki (omar.elmalki@epfl.ch)

### Semantic Role Labling on first 10 resolved ROCStories

In [1]:
from typing import List
import re
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

from allennlp.common import JsonDict
# Semantic Role Labeling with BERT : https://github.com/Riccorl/transformer-srl
from transformer_srl import dataset_readers, models, predictors

In [2]:
def sentence_to_srl(sentence: str) -> JsonDict:
    """
    Extracts Semantic Roles from a sentence.

    :param sentence: sentence from which to extract semantic roles labels.
    :return: semantic_roles as PropBank English SRLs.
    """

    # Pre-trained model with BERT fine-tuned to predict PropBank SRLs on CoNLL 2012 dataset.
    predictor = predictors.SrlTransformersPredictor.from_path(
        "../data/pre-trained-transformer-srl/srl_bert_base_conll2012.tar.gz", "transformer_srl")

    # More documentation: https://docs.allennlp.org/models/main/models/structured_prediction/predictors/srl/
    semantic_roles: JsonDict = predictor.predict(sentence)
    return semantic_roles

### Only keep entities

In [3]:
def srl_to_entities(srl: JsonDict):
    """
    Extract Knowledge triples from semantic roles

    :param srl: PropBank English SRLs
    :return: knowledge triples as a List of Lists
    """
    res = []
    verbs = srl['verbs']
    n_empty = 0
    for d in verbs:
        tags = d['tags']
        triple = d['description']
        verb = d['verb']
        args = re.findall('\\[ARG[01234]+.*?: (.*?)\\]', triple)
        for arg1 in args:  
            res.append(arg1)
    if not res:
        print(f"Empty entities for SRL: \n{srl}\n")
    return res

In [4]:
# Read ROCStories into pandas DataFrame
roc_stories_path_csv = '../generated/coreference_resolution/ROCStories_with_resolved_coreferences_first10.csv'
roc_stories_df = pd.read_csv(roc_stories_path_csv, sep='\t', header=0)

In [5]:
srl_entities_df = roc_stories_df

In [7]:
# Add Knowledge triples to Dataframe for each sentence in the dataset
for n in range(1,6):
    srl_entities_df[f'srl_entities{n}'] = srl_entities_df[f'resolved{n}'].progress_apply(lambda s: srl_to_entities(sentence_to_srl(s)))

 40%|████      | 4/10 [00:26<00:42,  7.10s/it]

Empty entities for SRL: 
{'verbs': [], 'words': ['a', 'pair', 'of', 'khakis', 'fit', 'Marcus', 'perfectly', '.']}



100%|██████████| 10/10 [01:30<00:00,  9.03s/it]


In [8]:
srl_entities_df

,Unnamed: 0,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5,coref_clusters,resolved1,resolved2,resolved3,resolved4,resolved5,srl_entities1,srl_entities2,srl_entities3,srl_entities4,srl_entities5
0,0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight recently.,He examined his habits to try and figure out the reason.,He realized he'd been eating too much fast food lately.,He stopped going to burger places and started a vegetarian diet.,"After a few weeks, he started to feel much better.","[David: [David, he, He, his, He, he, He, he]]",David noticed David had put on a lot of weight recently.,David examined David habits to try and figure out the reason.,David realized David'd been eating too much fast food lately.,David stopped going to burger places and started a vegetarian diet.,"After a few weeks, David started to feel much better.","[David, David had put on a lot of weight recently, David, a lot of weight]","[David, David habits, David, David, the reason]","[David, David'd been eating too much fast food lately, David'd, too much fast food]","[David, going to burger places, David, to burger places, David, a vegetarian diet]","[to feel much better, David, much better]"
1,1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about his actions.,"[Tom: [Tom, him, He, his, Tom, Tom, his, his]]",Tom had a very short temper.,One day a guest made Tom very angry.,Tom punched a hole in the wall of Tom house.,Tom's guest became afraid and left quickly.,Tom sat on Tom couch filled with regret about Tom actions.,"[Tom, a very short temper]","[a guest, Tom very angry]","[Tom, a hole in the wall of Tom house]","[Tom 's guest, afraid, Tom 's guest]","[Tom, on Tom couch, Tom, with regret about Tom actions]"
2,2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual event.,All of his clothes were either too formal or too casual.,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for the event.,"[Marcus: [Marcus, his, He, he, him, Marcus], a pair of khakis: [a pair of khakis, The pair he bought]]",Marcus needed clothing for a business casual event.,All of Marcus clothes were either too formal or too casual.,Marcus decided to buy a pair of khakis.,a pair of khakis fit Marcus perfectly.,Marcus was happy to have the right clothes for the event.,"[Marcus, clothing]","[All of Marcus clothes, either too formal or too casual]","[Marcus, to buy a pair of khakis, Marcus, a pair of khakis]",[],"[Marcus, happy to have the right clothes for the event, Marcus, the right clothes for the event]"
3,3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and haul it with his car.,Bill thought a truck would be better for what he needed.,Bobby pointed out two vehicles were much more expensive.,Bill was set in his ways with conventional thinking.,He ended up buying the truck he wanted despite Bobby's advice.,"[Bobby: [Bobby, Bobby, Bobby], Bill: [Bill, his, Bill, he, Bill, his, He, he], a trailer: [a trailer, it], a truck: [a truck, the truck he wanted]]",Bobby thought Bill should buy a trailer and haul a trailer with Bill car.,Bill thought a truck would be better for what Bill needed.,Bobby pointed out two vehicles were much more expensive.,Bill was set in Bill ways with conventional thinking.,Bill ended up buying a truck despite Bobby's advice.,"[Bobby, Bill should buy a trailer and haul a trailer with Bill car, Bill, buy a trailer, Bill, a trailer, Bill, a trailer]","[Bill, a truck would be better for what Bill needed, a truck, better for what Bill needed, what, Bill]","[Bobby, two vehicles were much more expensive, two vehicles, much more expensive]",

In [9]:
# Convert Dataframe to csv
srl_entities_df.to_csv('../generated/semantic-role-labeling/ROCStories_resolved_with_entities_first10.csv', sep='\t')